In [ ]:
'''_In this script, we ..._'''

In [ ]:
# -*- coding: utf-8 -*- #
# Authors: gracecaj@stud.ntnu.no, sbaarak@stud.ntnu.no, & annakale@stud.ntnu.no
# Date: 19/09/2023

In [1]:
#%%
#* Import the code needed
import os
from pathlib import Path
import numpy as np
import sys
import pandas as pd

In [2]:
#%%
#* Define paths
directory_path = Path('../../')

# Import data needed (pop, ppd, type split)

In [22]:
pop = pd.read_excel(
    directory_path / 'data' / 'regression_data' / 'Population' / 'regression_Population.xlsx',
    index_col=0)

PpD = pd.read_excel(
    directory_path / 'data' / 'regression_data' / 'PpD' / 'regression_PpD.xlsx',
    index_col=0)

TS_cj = pd.read_excel(
    directory_path / 'data' / 'cleaned_and_combined_data' / 'Type_Split' / 'Type_Split_ratios.xlsx',
    index_col=0)

lifetime = pd.read_excel(
    directory_path / 'data' / 'regression_data' / 'Dwelling_Lifetime' / 'lifetime_regression.xlsx',
    index_col=0)

lifetime_standard_deviation = pd.read_excel(
    directory_path / 'data' / 'regression_data' / 'Dwelling_Lifetime' / 'lifetime_regression.xlsx',
    index_col=0)

lifetime = lifetime.drop('std_dev', axis=1)
lifetime_standard_deviation = lifetime_standard_deviation.drop('lifetime', axis=1)

lifetime = lifetime.loc[:, 'lifetime'].to_numpy()
lifetime

lifetime_standard_deviation = lifetime_standard_deviation.loc[:, 'std_dev'].to_numpy()

#lifetime = lifetime.transpose()
#lifetime_standard_deviation = lifetime_standard_deviation.transpose()

# Import DSM model


In [4]:
# getting the name of the directory where the this file is present.
current = os.path.dirname(os.path.realpath('this'))
 
# Getting the parent directory name where the current directory is present.
parent = os.path.dirname(current)
#make the UA path
module_path = parent + '/modules'
#append path to options to import from
sys.path.append(module_path)

from dynamic_stock_model import DynamicStockModel

# Use pop and PpD to calculate stock over time

In [5]:
stock_t = np.multiply(pop,PpD)

stock_t_SFH = np.multiply(stock_t,TS_cj['SFH_ratio'])
stock_t_TH = np.multiply(stock_t,TS_cj['TH_ratio'])
stock_t_AB = np.multiply(stock_t,TS_cj['AB_ratio'])

C:\Users\grace\AppData\Local\Temp\ipykernel_560\669566578.py:1: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  stock_t = np.multiply(pop,PpD)
C:\Users\grace\AppData\Local\Temp\ipykernel_560\669566578.py:3: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 

In [10]:
stock_t_SFH

,Population
Year,
1600,1.129272e+06
1601,1.137121e+06
1602,1.144676e+06
1603,1.151944e+06
1604,1.158930e+06
...,...
2046,5.985500e+06
2047,5.980945e+06
2048,5.975777e+06


In [88]:
dwellings_stock_driven = DynamicStockModel(t=np.arange(1600, 2051),
                         s=stock_t_SFH,
                         lt={'Type': 'Normal', 
                             'Mean': [150],
                             'StdDev':[15] 
                             }
                        )

#len(stock_t_SFH.iloc[0]) == len(dwellings_stock_driven.t)

len(stock_t_SFH.iloc[0])
#len(dwellings_stock_driven.t)

inflow_tc = dwellings_stock_driven.compute_i_c_from_s_c(stock_t_SFH.iloc[:,0])
#print(lifetime)
#print(stock_t_SFH.iloc[:,0])
print(dwellings_stock_driven.lt['Mean'][0])
#print(dwellings_stock_driven.lt['Mean'][1][0])

KeyError: 0

In [17]:
type(stock_t_SFH['Population'].to_numpy())

numpy.ndarray

In [27]:
lifetime_test = 150
std_dev_test = 15

#cohorts = 

dwellings_stock_driven = DynamicStockModel(t=np.arange(1600, 2051),
                         s=stock_t_SFH['Population'].to_numpy(),
                         #s_c = cohorts
                         lt={'Type': 'Normal', 
                             'Mean': lifetime,
                             'StdDev': lifetime_standard_deviation 
                             }
                        )

# Calculate the stock by cohort
dwellings_stock_driven.compute_stock_driven_model(NegativeInflowCorrect = True) 

# Calculate  total outflow
dwellings_stock_driven.compute_outflow_total()

# Calculate the total stock
dwellings_stock_driven.compute_stock_total()

# Calculate the inflow
inflow_t = dwellings_stock_driven.i

# Calculate the outflow
outflow_t = dwellings_stock_driven.o

# Calculate the stock model
stock_tc = dwellings_stock_driven.s_c

i_c = dwellings_stock_driven.i_c

dwellings_stock_driven.dimension_check()

'<br><b> Checking dimensions of dynamic stock model DSM.Time vector is present with 451 years.<br>Inflow vector is present with 451 years.<br>Total stock is present with 451 years.<br>Stock by cohorts is present with 451 years and 451 cohorts.<br>Total outflow is present with 451 years.<br>Outflow by cohorts is present with 451 years and 451 cohorts.<br>Lifetime distribution is present with type Normal and mean [186.67 186.48 186.3  186.12 185.93 185.75 185.57 185.38 185.2  185.02\n 184.83 184.65 184.47 184.28 184.1  183.92 183.73 183.55 183.37 183.18\n 183.   182.82 182.63 182.45 182.27 182.08 181.9  181.72 181.53 181.35\n 181.17 180.98 180.8  180.62 180.43 180.25 180.07 179.88 179.7  179.52\n 179.33 179.15 178.97 178.78 178.6  178.42 178.23 178.05 177.87 177.68\n 177.5  177.32 177.13 176.95 176.77 176.58 176.4  176.22 176.03 175.85\n 175.67 175.48 175.3  175.12 174.93 174.75 174.57 174.38 174.2  174.02\n 173.83 173.65 173.47 173.28 173.1  172.92 172.73 172.55 172.37 172.18\n 172.   1

In [29]:
print(i_c)

None


In [26]:
dwellings_stock_driven.dimension_check()

'<br><b> Checking dimensions of dynamic stock model DSM.Time vector is present with 451 years.<br>Inflow vector is present with 451 years.<br>Total stock is present with 451 years.<br>Stock by cohorts is present with 451 years and 451 cohorts.<br>Total outflow is present with 451 years.<br>Outflow by cohorts is present with 451 years and 451 cohorts.<br>Lifetime distribution is present with type Normal and mean [186.67 186.48 186.3  186.12 185.93 185.75 185.57 185.38 185.2  185.02\n 184.83 184.65 184.47 184.28 184.1  183.92 183.73 183.55 183.37 183.18\n 183.   182.82 182.63 182.45 182.27 182.08 181.9  181.72 181.53 181.35\n 181.17 180.98 180.8  180.62 180.43 180.25 180.07 179.88 179.7  179.52\n 179.33 179.15 178.97 178.78 178.6  178.42 178.23 178.05 177.87 177.68\n 177.5  177.32 177.13 176.95 176.77 176.58 176.4  176.22 176.03 175.85\n 175.67 175.48 175.3  175.12 174.93 174.75 174.57 174.38 174.2  174.02\n 173.83 173.65 173.47 173.28 173.1  172.92 172.73 172.55 172.37 172.18\n 172.   1

In [25]:
print(inflow_t)

[1.13081922e+06 7.94314512e+03 7.65415795e+03 7.37085223e+03
 7.09323993e+03 6.82131804e+03 6.55509886e+03 6.29459431e+03
 6.03980313e+03 5.79073759e+03 5.54740960e+03 5.30981958e+03
 5.07797982e+03 4.85190227e+03 4.63158896e+03 4.41705221e+03
 4.20830406e+03 4.00535638e+03 3.80820486e+03 3.61687009e+03
 3.43135696e+03 3.25167803e+03 3.07784562e+03 2.90986594e+03
 2.74775167e+03 2.59151529e+03 2.44116420e+03 2.29671123e+03
 2.15816903e+03 2.02554605e+03 1.89885526e+03 1.77810947e+03
 1.66331808e+03 1.55449417e+03 1.45165068e+03 1.35479782e+03
 1.26394876e+03 1.17911655e+03 1.10031204e+03 1.02754844e+03
 9.60838884e+02 9.00194684e+02 8.45629065e+02 7.97155153e+02
 7.54784571e+02 7.18530469e+02 6.88405909e+02 6.64422635e+02
 6.46593642e+02 6.34931837e+02 6.29448891e+02 6.30157547e+02
 6.37070460e+02 6.50200249e+02 6.69556932e+02 6.95153950e+02
 7.27002194e+02 7.65113568e+02 8.09499871e+02 8.60171271e+02
 9.17139056e+02 9.80414401e+02 1.05000651e+03 1.12592593e+03
 1.20818304e+03 1.296785

# Enter stock into stock driven model to get inflows by cohort over time

In [47]:
from dynamic_stock_model import DynamicStockModel

#! The following function and runs of it should effectively iterate things over for each of the types

def stock_model(stock):
    dwellings_stock_driven = DynamicStockModel(t=np.arange(1600, 2051),
                         s=stock,
                         lt={'Type': 'Normal', 
                             'Mean': lifetime,
                             'StdDev':lifetime_standard_deviation
                             }
                        )
    #inflow_tc = dwellings_stock_driven.i_c

    dwellings_stock_driven.compute_stock_driven_model(NegativeInflowCorrect = True)
    inflow_t = dwellings_stock_driven.i
    

     
    inflow_tcj = np.multiply(inflow_tc,TS_cj)
    
    dwellings_inflow_driven = DynamicStockModel(t=np.arange(1600, 2051),
                        i=inflow_tcj,
                        lt={'Type': 'Normal', 
                            'Mean': lifetime,
                            'StdDev': lifetime_standard_deviation 
                            }
                        )
    dwellings_inflow_driven.compute_s_c_inflow_driven()
    return dwellings_inflow_driven.s_c

s_c_SFH = stock_model(stock_t_SFH)
s_c_TH = stock_model(stock_t_TH)
s_c_AB = stock_model(stock_t_AB)

#! make some kind of stacked line chart with this. also compute sum

s_c_total = s_c_SFH + s_c_TH + s_c_AB




#! I'm not sure if we can use a vector of different lifetime values or if there can only be one constant. Check DSM module

dwellings_stock_driven = DynamicStockModel(t=np.arange(1600, 2050),
                         s=stock_t,
                         lt={'Type': 'Normal', 
                             'Mean': [lifetime],
                             'StdDev':[lifetime_standard_deviation] 
                             }
                        )

#! I think the following will work but I'm not 100% sure
# Calculate the inflow
inflow_tc = dwellings_stock_driven.i_c

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

# multiply inflow by type split to get inflow by type split and cohort

In [ ]:
inflow_tcj = np.multiply(inflow_tc,TS_cj)

# enter inflow by type split and cohort into inflow driven model to get stock by cohort and type split over time

In [ ]:
# Define our dynamic stock model using our data and parameters
dwellings_inflow_driven = DynamicStockModel(t=np.arange(1600, 2050),
                        i=inflow_tcj,
                        lt={'Type': 'Normal', 
                            'Mean': np.array([lifetime]),
                            'StdDev': np.array([lifetime_standard_deviation]) 
                            }
                        )


#! I think the following will work but I'm not 100% sure
# Calculate the stock by cohort
dwellings_inflow_driven.compute_s_c_inflow_driven()
s_c = dwellings_inflow_driven.s_c
